# Data Cleansing and Preparation:
## The initial data cleansing and preparation steps were followed by the work in the source below:
### https://www.kaggle.com/code/okanulu/android-malware-prediction-with-ml-algorithms
## The choice to use only Android_SMS_Malware and Benign labels along with the Support Vector classifier Machine model implementation, analysis, and evaluation are uniue to this notebook.t

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.exceptions import ConvergenceWarning, UndefinedMetricWarning
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix
import warnings

In [10]:
data = pd.read_csv("../../../data/Android_Malware.csv")
print(data)
data = data.drop(' Timestamp', axis=1)

C:\Users\grace\AppData\Local\Temp\ipykernel_14056\4204408830.py:1: DtypeWarning: Columns (56,58,63) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../../../data/Android_Malware.csv")


        Unnamed: 0                                  Flow ID     Source IP  \
0                0    172.217.6.202-10.42.0.211-443-50004-6   10.42.0.211   
1                1    172.217.6.202-10.42.0.211-443-35455-6   10.42.0.211   
2                2    131.253.61.68-10.42.0.211-443-51775-6   10.42.0.211   
3                3    131.253.61.68-10.42.0.211-443-51775-6   10.42.0.211   
4                4    131.253.61.68-10.42.0.211-443-51776-6   10.42.0.211   
...            ...                                      ...           ...   
355625         405      172.217.7.14-10.42.0.211-80-38405-6  172.217.7.14   
355626         406         10.42.0.211-10.42.0.1-7632-53-17   10.42.0.211   
355627         407  10.42.0.211-104.192.110.245-45970-443-6   10.42.0.211   
355628         408        10.42.0.211-10.42.0.1-51982-53-17   10.42.0.211   
355629         409         10.42.0.211-10.42.0.1-9320-53-17   10.42.0.211   

         Source Port   Destination IP   Destination Port   Protocol  \
0   

In [12]:
print(data.columns.tolist())

['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', 

In [14]:
print(data['Label'].value_counts())
names=sorted(data['Label'].unique().tolist())
print(names)

Label
Android_Adware         147443
Android_Scareware      117082
Android_SMS_Malware     67397
Benign                  23708
Name: count, dtype: int64
['Android_Adware', 'Android_SMS_Malware', 'Android_Scareware', 'Benign']


In [16]:
print(data.dtypes)

Unnamed: 0           int64
Flow ID             object
 Source IP          object
 Source Port         int64
 Destination IP     object
                    ...   
Idle Mean          float64
 Idle Std          float64
 Idle Max          float64
 Idle Min          float64
Label               object
Length: 85, dtype: object


In [18]:
data

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,172.217.6.202-10.42.0.211-443-50004-6,10.42.0.211,50004,172.217.6.202,443.0,6.0,37027,1,1,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
1,1,172.217.6.202-10.42.0.211-443-35455-6,10.42.0.211,35455,172.217.6.202,443.0,6.0,36653,1,1,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
2,2,131.253.61.68-10.42.0.211-443-51775-6,10.42.0.211,51775,131.253.61.68,443.0,6.0,534099,8,12,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
3,3,131.253.61.68-10.42.0.211-443-51775-6,10.42.0.211,51775,131.253.61.68,443.0,6.0,9309,3,0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
4,4,131.253.61.68-10.42.0.211-443-51776-6,10.42.0.211,51776,131.253.61.68,443.0,6.0,19890496,8,6,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355625,405,172.217.7.14-10.42.0.211-80-38405-6,172.217.7.14,80,10.42.0.211,38405.0,6.0,126711,1,1,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
355626,406,10.42.0.211-10.42.0.1-7632-53-17,10.42.0.211,7632,10.42.0.1,53.0,17.0,48012,1,1,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
355627,407,10.42.0.211-104.192.110.245-45970-443-6,10.42.0.211,45970,104.192.110.245,443.0,6.0,20028018,11,8,...,20.0,367528.0,0.0,367528.0,367528.0,19660490.0,0.0,19660490.0,19660490.0,Benign
355628,408,10.42.0.211-10.42.0.1-51982-53-17,10.42.0.211,51982,10.42.0.1,53.0,17.0,347926,1,1,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [22]:
print("Total Features : ",len(data.columns)-1)

Total Features :  84


# Data Preprocessing done unique to this notebook:

- Filtering the Dataset: The dataset is filtered to include only two labels—'Benign' and 'Android_SMS_Malware'—by dropping irrelevant categories ('Android_Adware' and 'Android_Scareware'). This ensures the model focuses on the most relevant classification task.

- Dropping Non-Numeric Columns: Non-numeric columns, such as 'Flow ID', 'Source IP', 'Destination IP', and others, are removed from the feature set. This is critical because machine learning models, including Random Forests, require numerical input data for training.

- Ensuring Numeric Data: After dropping non-numeric columns, the feature matrix (X) is checked to ensure only numeric columns remain. This step ensures that there are no data type issues when passing the data to the model.

- Label Encoding: The target variable (y) is mapped to integers—0 for 'Benign' and 1 for 'Android_SMS_Malware'. This allows the model to interpret the categorical labels as numeric values for classification.

## Step 1: Filter the dataset to include only 'Android_SMS_Malware' and 'Benign'

In [26]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.utils import resample

filtered_data = data[data["Label"].isin(['Android_SMS_Malware', 'Benign'])]

## Step 2: Drop non-numeric columns

In [29]:
X_filtered = filtered_data.drop(['Flow ID', ' Source IP', ' Destination IP', ' CWE Flag Count', ' Down/Up Ratio', 'Fwd Avg Bytes/Bulk', 'Label'], axis=1)

## Step 3: Ensure only numeric data remains

In [32]:
X_filtered_numeric = X_filtered.select_dtypes(include=[np.number])

## Verify that only numeric columns remain

In [35]:
print(X_filtered_numeric.dtypes)

Unnamed: 0             int64
 Source Port           int64
 Destination Port    float64
 Protocol            float64
 Flow Duration         int64
                      ...   
 Active Min          float64
Idle Mean            float64
 Idle Std            float64
 Idle Max            float64
 Idle Min            float64
Length: 78, dtype: object


## Step 4: Separate features (X) and target (y) after filtering

In [38]:
X = X_filtered_numeric
y = filtered_data["Label"]

## Step 5: Convert the target labels into numerical categories (Benign = 0, Android_SMS_Malware = 1)

In [41]:
y = y.map({'Benign': 0, 'Android_SMS_Malware': 1})

## Step 6: Check the data types of y to ensure it's numeric

In [44]:
print(y.dtypes)  # It should print "int64" or similar to indicate discrete values

int64


# Data Balancing: 
- Balancing Classes: Since class imbalance can negatively impact model performance, the balance_data function resamples the data to ensure both classes ('Benign' and 'Android_SMS_Malware') are equally represented. This step involves resampling the minority class (whichever has fewer samples) so that both classes have the same number of data points. After balancing, the data is shuffled to randomize the sample order.

- Ensuring Numeric Labels: The balanced target variable (y_balanced) is explicitly converted to integers. This step ensures that the model correctly interprets the labels as discrete classes during training and evaluation.

## Step 7: Balance the dataset using the balance_data function

In [48]:
def balance_data(X, y, random_state=0):
    # Combine X and y into one DataFrame
    data = pd.concat([X, y], axis=1)
    class_counts = y.value_counts()
    minority_class_count = class_counts.min()
    balanced_data = pd.DataFrame(columns=data.columns)

    for label in class_counts.index:
        class_samples = data[data["Label"] == label]
        resampled_samples = resample(class_samples,
                                     replace=True,
                                     n_samples=minority_class_count,
                                     random_state=random_state)
        balanced_data = pd.concat([balanced_data, resampled_samples])

    balanced_data = balanced_data.sample(frac=1, random_state=random_state).reset_index(drop=True)
    return balanced_data.drop("Label", axis=1), balanced_data["Label"]

## Step 8: Balance the data

In [51]:
X_balanced, y_balanced = balance_data(X, y)

C:\Users\grace\AppData\Local\Temp\ipykernel_14056\3078144896.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  balanced_data = pd.concat([balanced_data, resampled_samples])


## Step 9: Ensure y_balanced is correctly mapped to integers

In [54]:
y_balanced = y_balanced.astype(int)
print(y_balanced.dtypes)  

int32


## Step 10: Check for and remove missing values
- Checks for NaN (missing) values in each column of the original dataset (X_balanced) before handling missing values.
- Uses the SimpleImputer to impute (fill in) missing values in the dataset by replacing NaNs with the mean value of the respective column.
- Converts the imputed data (initially a NumPy array) back into a DataFrame to maintain compatibility with the rest of the code.
- Verifies that all NaN values have been successfully imputed by checking for any remaining NaNs in the cleaned dataset (X_balanced_clean).
- Ensures that the target variable (y_balanced) remains aligned with the cleaned feature dataset (X_balanced_clean).

In [73]:
# Check for NaN values before imputation
print("Number of NaN values in each column before imputation:")
print(X_balanced.isna().sum())

# Impute missing values using the mean strategy
imputer = SimpleImputer(strategy='mean')
X_balanced_clean = imputer.fit_transform(X_balanced)

# Convert the cleaned data back into a DataFrame, if needed
X_balanced_clean = pd.DataFrame(X_balanced_clean, columns=X_balanced.columns)

# Check for NaN values after imputation
print("Number of NaN values in the cleaned dataset after imputation:")
print(np.isnan(X_balanced_clean).sum().sum())  # sum().sum() to check total NaN count

# Ensure y_balanced is aligned with the cleaned X
y_balanced_clean = y_balanced

Number of NaN values in each column before imputation:
Unnamed: 0           0
 Source Port         0
 Destination Port    0
 Protocol            0
 Flow Duration       0
                    ..
 Active Min          2
Idle Mean            2
 Idle Std            2
 Idle Max            2
 Idle Min            2
Length: 78, dtype: int64
Number of NaN values in the cleaned dataset after imputation:
0


# Cross-Validation and Classification Metrics Analysis
This code performs 10-fold cross-validation on a dataset using various configurations of Support Vector Classifier (SVC) models, evaluating their performance with accuracy, confusion matrix, and classification report.

## 1. SVC Configurations:
- Linear Kernel SVC: Assumes a linear boundary between classes in the feature space.
- RBF Kernel SVC: Uses the Radial Basis Function to capture non-linear boundaries between classes by mapping features into a higher-dimensional space.
- Polynomial Kernel SVC: Uses polynomial decision boundaries, allowing the model to capture more complex class relationships.

## 10-fold Cross-Validation: The `cross_val_predict` function is used to perform 10-fold cross-validation on the balanced dataset. In this process:
- The dataset is split into 10 equal subsets (folds).
- For each fold, the model is trained on 9 subsets and evaluated on the remaining subset.
- This process is repeated across all 10 folds, ensuring that every data point is used for both training and evaluation.
- Cross-validation helps mitigate overfitting by providing a robust evaluation of the model’s ability to generalize to unseen data.

## Evaluation Metrics:
- `cross_val_score`: This function is used to evaluate the accuracy of each fold, and the results are printed along with the average accuracy across all folds.
- Accuracy: The overall accuracy of the model based on the combined predictions from the cross-validation is calculated and reported.
- Confusion Matrix: A confusion matrix is generated to display how well the model predicts each class (e.g., 'Benign' vs. 'Android_SMS_Malware').
- Classification Report: A classification report provides detailed metrics such as precision, recall, F1-score, and support for each class, offering insights into how well the model distinguishes between 'Benign' and 'Android_SMS_Malware'.

## Conclusion:
By using balanced data and evaluating various SVC models with 10-fold cross-validation, this approach provides a comprehensive and reliable evaluation of each model's performance. The results help in identifying the most effective SVC configuration for classifying 'Benign' and 'Android_SMS_Malware' traffic, ensuring robust generalization on new data.

In [81]:
from sklearn.svm import SVC  # Import Support Vector Classifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, cross_val_predict

# Define the SVC models with different kernels and regularization (C) values
models = [
   ('SVC with linear kernel', SVC(kernel='linear', C=1, random_state=0)),
    ('SVC with RBF kernel', SVC(kernel='rbf', C=1, gamma=0.1, random_state=0)),
    ('SVC with polynomial kernel', SVC(kernel='poly', degree=3, C=1, random_state=0))
]

# Function to evaluate the model using cross-validation
def evaluate_model_cv(model, X, y, model_name):
    # Get predictions using cross-validation
    y_pred = cross_val_predict(model, X, y, cv=10, n_jobs=-1)
    
    # Calculate accuracy
    accuracy = accuracy_score(y, y_pred)
    
    # Generate confusion matrix
    confusion_matrix_result = confusion_matrix(y, y_pred)
    
    # Generate classification report
    classification = classification_report(y, y_pred)

    # Perform 10-fold cross-validation to get accuracy for each fold
    scores = cross_val_score(model, X, y, cv=10, scoring='accuracy', n_jobs=-1)
    
    # Print evaluation results
    print(f'============================== {model_name} Cross-Validation Evaluation ==============================')
    
    # Print accuracy for each fold and the mean accuracy
    print(f"Accuracy for each fold: {scores}")
    print(f"Mean Accuracy across folds: {np.mean(scores):.4f}")
    print("\n")
    
    # Print overall evaluation using combined cross-validation predictions
    print(f'Model Accuracy (combined prediction): {accuracy * 100:.2f}%')
    print('Confusion Matrix:\n', confusion_matrix_result)
    print('Classification Report:\n', classification)
    print()

# Iterate through each SVC model and evaluate using cross-validation
for model_name, model in models:
    evaluate_model_cv(model, X_balanced_clean, y_balanced_clean, model_name)

KeyboardInterrupt: 

# Save output in a code block to preserve the format of the output

In [85]:
import seaborn as sns
Plot confusion matrix (optional but helpful for visualization)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

SyntaxError: invalid syntax (383072502.py, line 2)